In [ ]:
# LLM 모델을 통한 연산 예제
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel,Field

llm = ChatOpenAI(temperature=0)


# a와 b의 형태 지정
class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")

# BaseTool을 사용하여 직접 Tool 형식을 지정
class CalculatorTool(BaseTool):
    # 툴 이름
    name = "SumsCalculator"
    # 툴에 대한 설명 작성(프롬프트)
    description="""
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    # 함수 인자 스키마 지정(입력 값 형태)
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b

    

# 에이전트 생성
agent = initialize_agent(
    # 사용할 LLM 모델
    llm=llm,

    # 추론 과정 디버깅(에전트가 하는 작업의 모든 과정 및 추론을 직접 확인해볼 수 있음)
    verbose = True, 

    # 에이전트 유형 설정
    agent=AgentType.OPENAI_FUNCTIONS,

    # LLM이 우리가 원하는 출력 형식으로 응답하지 않아서 파싱 에러가 발생하면 에이전트가 오류를 처리하고 대응함.
    handle_parsing_errors=True,

    # 에이전트가 사용할 도구 모음(Tools)
    tools=[
       CalculatorTool(),
    ]
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12."


# 이번엔 에이전트에게 프롬프트를 입력(위에선 LLM모델에 입력했음)
agent.invoke(prompt)